<a href="https://colab.research.google.com/github/Sikandarh11/Data-Science-Online-Notes/blob/main/hyperparameterTunnig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv("/content/diabetes.csv")

In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [7]:
X =  df.iloc[:, :-1].values
y = df['Outcome'].values

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)

In [9]:
scaled_data.shape

(768, 8)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_data,y, random_state = 42, test_size=0.2)

In [11]:
import tensorflow
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [12]:
model = Sequential()
model.add(Dense(32, activation = "relu", input_dim=8))
model.add(Dense(1, activation="sigmoid"))

In [13]:
model.compile(optimizer='Adam', loss="binary_crossentropy", metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
20/20 [==============================] - 1s 13ms/step - loss: 0.6753 - accuracy: 0.5814 - val_loss: 0.6662 - val_accuracy: 0.5909
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 0.6277 - accuracy: 0.6515 - val_loss: 0.6298 - val_accuracy: 0.6364
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5920 - accuracy: 0.6987 - val_loss: 0.6007 - val_accuracy: 0.6883
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5627 - accuracy: 0.7313 - val_loss: 0.5784 - val_accuracy: 0.7013
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5405 - accuracy: 0.7394 - val_loss: 0.5602 - val_accuracy: 0.7143
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5228 - accuracy: 0.7524 - val_loss: 0.5475 - val_accuracy: 0.7338
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5083 - accuracy: 0.7590 - val_loss: 0.5352 - val_accuracy: 0.7468
Epoch 8/10
20/20 [=

In [15]:
# 1. How to select approriate optimizer
# 2. How to select no. of layers
# 3. No. of nodes
# 4. All in one model

In [16]:
pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.2 MB/s eta 0:00:00


In [17]:
import kerastuner as kt

<ipython-input-17-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [18]:
def build_model1(hp):
  model = Sequential()
  model.add(Dense(32,activation="relu", input_dim=8))
  model.add(Dense(1,activation="sigmoid"))
  optimizers=hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta'])
  model.compile(optimizer=optimizers, loss = 'binary_crossentropy', metrics=['accuracy'])
  return model

In [20]:
tuner = kt.RandomSearch(build_model1,
                        objective = "val_accuracy",
                        max_trials=5)

In [21]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 01s]
val_accuracy: 0.6688311696052551

Best val_accuracy So Far: 0.7597402334213257
Total elapsed time: 00h 00m 06s


In [22]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [23]:
model = tuner.get_best_models(num_models =1)[0]

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
model.fit(X_train, y_train, epochs=100,initial_epoch=5, validation_data=(X_test, y_test))

Epoch 6/100
20/20 [==============================] - 1s 11ms/step - loss: 0.5366 - accuracy: 0.7345 - val_loss: 0.5105 - val_accuracy: 0.7532
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5211 - accuracy: 0.7443 - val_loss: 0.5012 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5087 - accuracy: 0.7508 - val_loss: 0.4938 - val_accuracy: 0.7597
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4991 - accuracy: 0.7573 - val_loss: 0.4905 - val_accuracy: 0.7532
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4903 - accuracy: 0.7590 - val_loss: 0.4879 - val_accuracy: 0.7468
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4833 - accuracy: 0.7573 - val_loss: 0.4859 - val_accuracy: 0.7597
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4775 - accuracy: 0.7687 - val_loss: 0.4868 - val_accuracy: 0.7532
Epoch 13/

# **# of neuron in this layer**

In [32]:
def build_model(hp):
  model = Sequential()
  units = hp.Int('units', 8,128, step=8)
  model.add(Dense(units=units, activation='relu', input_dim=8))
  model.add(Dense(units=1, activation='sigmoid'))

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [33]:
tuner1= kt.RandomSearch(
    build_model1,
    objective=  'val_accuracy',
    max_trials=5
    )

Reloading Tuner from ./untitled_project/tuner0.json


In [34]:
tuner1.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [35]:
tuner1.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [49]:
def build_model(hp):
    model = Sequential()
    units = hp.Int('units', min_value=8, max_value=128)
    model.add(Dense(units=units, activation='relu', input_dim=8))
    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Initialize the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,  # Optionally add this to run multiple executions per trial
    directory='my_dir',
    project_name='helloworld'
)

# Search for the best hyperparameters
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))



Reloading Tuner from my_dir/helloworld/tuner0.json


In [50]:
best_hyperparameters = tuner.get_best_hyperparameters()[0]
print(best_hyperparameters.values)

{'units': 128}


In [51]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1152      
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1281 (5.00 KB)
Trainable params: 1281 (5.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
model.fit(X_train, y_train, epochs=100, initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 1s 2ms/step - loss: 0.4721 - accuracy: 0.7769
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4610 - accuracy: 0.7720
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4539 - accuracy: 0.7720
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4487 - accuracy: 0.7736
Epoch 11/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4448 - accuracy: 0.7785
Epoch 12/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4398 - accuracy: 0.7752
Epoch 13/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4390 - accuracy: 0.7834
Epoch 14/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4337 - accuracy: 0.7850
Epoch 15/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4327 - accuracy: 0.7834
Epoch 16/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4306 - accuracy: 0.7850
Epo

In [48]:
#how to select the number of layers

In [66]:
def build_model2(hp):
  model = Sequential()
  model.add(Dense(72, activation='relu', input_dim=8))
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(1, activation='relu'))
  model.add(Dense(activation='sigmoid'))
  model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['acuracy'])
  return model

In [67]:
tuner = kt.RandomSearch(
    build_model2,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,  # Optionally add this to run multiple executions per trial
    directory='project2',
    project_name='noOfLayres'
)

Reloading Tuner from project2/noOfLayres/tuner0.json


In [68]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [69]:
tuner.get_best_hyperparameters()[0].values

{'units': 64}

In [72]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(72, activation='relu', input_dim=8))
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        model.add(Dense(1, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Corrected this line
    model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])  # Corrected this line
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='project2',
    project_name='noOfLayres'
)

tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

best_hyperparameters = tuner.get_best_hyperparameters()[0]
print(best_hyperparameters.values)


Reloading Tuner from project2/noOfLayres/tuner0.json
{'units': 64}


In [77]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def build_model2(hp):
    model = Sequential()
    # Hyperparameter for number of units in the first layer
    #units = hp.Int('units', min_value=8, max_value=128, step=8)
    model.add(Dense(units=72, activation='relu', input_dim=8))
    # Hyperparameter for number of additional layers
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        model.add(Dense(units=72, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])  # Corrected this line
    return model

tuner = kt.RandomSearch(
    build_model2,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='project2',
    project_name='noOfLayres'
)

tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))


Reloading Tuner from project2/noOfLayres/tuner0.json


In [78]:
best_hyperparameters = tuner.get_best_hyperparameters()[0]
print(best_hyperparameters.values)

{'units': 64}


In [81]:
import shutil
shutil.rmtree('project2/noOfLayres', ignore_errors=True)  # Clear previous tuner results


In [82]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_model2(hp):
    model = Sequential()
    # Hyperparameter for number of units in the first layer
    #units = hp.Int('units', min_value=8, max_value=128, step=8)
    model.add(Dense(units=72, activation='relu', input_dim=8))
    # Hyperparameter for number of additional layers
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        model.add(Dense(units=72, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(
    build_model2,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='project2',
    project_name='noOfLayres'
)

tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

best_hyperparameters = tuner.get_best_hyperparameters()[0]
print(best_hyperparameters.values)

# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Print the summary of the best model
best_model.summary()


Trial 5 Complete [00h 00m 08s]
val_accuracy: 0.7813852628072103

Best val_accuracy So Far: 0.7813852628072103
Total elapsed time: 00h 00m 40s
{'num_layers': 6}
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 72)                648       
                                                                 
 dense_1 (Dense)             (None, 72)                5256      
                                                                 
 dense_2 (Dense)             (None, 72)                5256      
                                                                 
 dense_3 (Dense)             (None, 72)                5256      
                                                                 
 dense_4 (Dense)             (None, 72)                5256      
                                                                 
 dense_5 (Dense)            

In [84]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6}

In [85]:
model=tuner.get_best_models(num_models=1)[0]

In [86]:
model.fit(X_train, y_train, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 [==============================] - 2s 14ms/step - loss: 0.4543 - accuracy: 0.7883 - val_loss: 0.5901 - val_accuracy: 0.7208
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4457 - accuracy: 0.7997 - val_loss: 0.5496 - val_accuracy: 0.7273
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4316 - accuracy: 0.7964 - val_loss: 0.5538 - val_accuracy: 0.7597
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4157 - accuracy: 0.8046 - val_loss: 0.5716 - val_accuracy: 0.7143
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3915 - accuracy: 0.8160 - val_loss: 0.5771 - val_accuracy: 0.7403
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3821 - accuracy: 0.8225 - val_loss: 0.6027 - val_accuracy: 0.7532
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3711 - accuracy: 0.8436 - val_loss: 0.6063 - val_accuracy: 0.7597
Epoch 14

In [87]:
import shutil
shutil.rmtree('project2/noOfLayres', ignore_errors=True)  # Clear previous tuner results


In [102]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
def build_model2(hp):
    model = Sequential()
    counter=0
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        if counter==0:
          model.add(
              Dense(
                  hp.Int("units" + str(i), min_value=8, max_value=128),
                  activation=hp.Choice("activation"+str(i), values=['relu', 'tanh', 'sigmoid']),
                  input_dim=8
                  ))
          model.add(Dropout(hp.Choice('dropout'+str(i), values=[.1,.2,.3,.4,.5,.6,.7,.8,.9])))
        else:
            model.add(
              Dense(
                  hp.Int("units" + str(i), min_value=8, max_value=128),
                  activation=hp.Choice("activation"+str(i), values=['relu', 'tanh', 'sigmoid']),
                  ))
            model.add(Dropout(hp.Choice('dropout'+str(i), values=[.1,.2,.3,.4,.5,.6,.7,.8,.9])))

        counter+=1
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['rmsprop', 'adam','sgd']),
        loss='binary_crossentropy',
        metrics=['accuracy']
                  )
    return model

In [103]:
tuner = kt.RandomSearch(
    build_model2,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='project2',
    project_name='final2'
)


In [104]:

tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

best_hyperparameters = tuner.get_best_hyperparameters()[0]
print(best_hyperparameters.values)


Trial 5 Complete [00h 00m 06s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6450216372807821
Total elapsed time: 00h 00m 34s
{'num_layers': 1, 'units0': 55, 'activation0': 'sigmoid', 'dropout0': 0.6, 'optimizer': 'sgd', 'units1': 18, 'activation1': 'sigmoid', 'dropout1': 0.9, 'units2': 9, 'activation2': 'tanh', 'dropout2': 0.2, 'units3': 52, 'activation3': 'tanh', 'dropout3': 0.7, 'units4': 14, 'activation4': 'sigmoid', 'dropout4': 0.1, 'units5': 20, 'activation5': 'sigmoid', 'dropout5': 0.9, 'units6': 41, 'activation6': 'tanh', 'dropout6': 0.6, 'units7': 22, 'activation7': 'tanh', 'dropout7': 0.8, 'units8': 102, 'activation8': 'tanh', 'dropout8': 0.3, 'units9': 12, 'activation9': 'relu', 'dropout9': 0.1}


In [105]:
best_model = tuner.get_best_models(num_models=1)[0]

In [106]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 55)                495       
                                                                 
 dropout (Dropout)           (None, 55)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 56        
                                                                 
Total params: 551 (2.15 KB)
Trainable params: 551 (2.15 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [1]:
model.fit(X_train, y_train, initial_epoch=5,epochs=100 ,validation_data=(X_test, y_test))

NameError: name 'model' is not defined